In [1]:
import os
import sys
import math
import logging
from pathlib import Path

%load_ext autoreload
%autoreload 2

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import seaborn as sns
sns.set_context("poster")
sns.set(rc={"figure.figsize": (16, 9.)})
sns.set_style("whitegrid")

import pandas as pd
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

logging.basicConfig(level=logging.INFO, stream=sys.stdout)

# Querying Google's BigQuery

Let's say we want to find out how many monthly downloads some project on [PyPI](https://pypi.org/) has.

In [2]:
# CONFIGURATION
gcp_project = "cf-model-298607"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/fwilhelm/.google-app-creds.json"

# SQL, the direct way

In [3]:
from google.cloud import bigquery

bqclient = bigquery.Client()

Our SQL query is directly defined as string and `bigquery.Client()` is used to execute. 

In [4]:
query_string = """
SELECT 
  DATE_TRUNC(DATE(timestamp), MONTH) AS `month`,
  COUNT(*) AS num_downloads,
FROM `bigquery-public-data.pypi.file_downloads`
WHERE file.project = 'pyscaffold'
    AND details.installer.name = 'pip'
    AND DATE(timestamp) BETWEEN DATE('2021-01-01') AND CURRENT_DATE()
GROUP BY `month`
ORDER BY `month`
"""

df = (
    bqclient.query(query_string)
    .result()
    .to_dataframe(create_bqstorage_client=True)
)

df.head()

,month,num_downloads
0,2021-01-01,229342
1,2021-02-01,120081
2,2021-03-01,289039
3,2021-04-01,265421
4,2021-05-01,117702


As easy as it looks, this directy way has several downsides especially if some parameters like for instance the projet `pyscaffold` needs to be define dynamically. In this case, most developers resort to string substitutions, for more complex queries even templating engines like [Jinja](https://jinja.palletsprojects.com/) is used.

The obvious downsides of this approach are that string generation is cumbersome, error-prone and only at execution time of the query you know for sure that your query is really syntactically correct. Also clean coding like structering complex queries into smaller subqueries that get composed dynamically is hardly possible with this approach. A more subtle downside is that  [SQL-injection attacks](https://en.wikipedia.org/wiki/SQL_injection) are quite easy as no [prepared statements](https://en.wikipedia.org/wiki/Prepared_statement) are used.

# SQL, the programmatic way

Using [SQLAlchemy](https://www.sqlalchemy.org/) with the [BigQuery](https://github.com/googleapis/python-bigquery-sqlalchemy) dialect, we can programmatically assemble our query directly in Python. This allows our IDE to do introspection, pointing out errors and allowing Python to inform us about misformed SQL queries before even executing them. SQLAlchemy also uses prepared statementes automatically to avoid SQL injection attacks.

In [5]:
from sqlalchemy import *
from sqlalchemy.sql import func as F
from sqlalchemy.engine import create_engine
from sqlalchemy.schema import *

engine = create_engine(f'bigquery://{gcp_project}') 

In [6]:
table = Table('bigquery-public-data.pypi.file_downloads', MetaData(bind=engine), autoload=True)
query = (select([F.date_trunc(F.date(column("timestamp")), text("month")).label("month"),
                 F.count("*").label("num_downloads")],
                from_obj=table)
         .where(
             and_(
                F.date(column("timestamp")).between("2021-01-01", F.current_date()),
                column("file.project") == "pyscaffold",
                column("details.installer.name") == "pip"
             )
         )
         .group_by("month")
         .order_by("month"))

At any time we can take a look at the prepared statement using `print`.

In [7]:
print(query)

SELECT date_trunc(date(`timestamp`), month) AS `month`, count(%(count_1:STRING)s) AS `num_downloads` 
FROM `bigquery-public-data.pypi.file_downloads` 
WHERE date(`timestamp`) BETWEEN %(date_1:STRING)s AND CURRENT_DATE AND `file`.`project` = %(file.project_1:STRING)s AND `details`.`installer`.`name` = %(details.installer.name_1:STRING)s GROUP BY `month` ORDER BY `month`


A low-level way of executing the query now, is to use the `engine` with:
```python
result = engine.execute(query)
```
Since in most cases, we want to deal with a dataframe, we can just use Pandas:

In [8]:
df = pd.read_sql(query, engine)

In [9]:
df.head()

,month,num_downloads
0,2021-01-01,229342
1,2021-02-01,120081
2,2021-03-01,289039
3,2021-04-01,265421
4,2021-05-01,117702
